In [1]:
import pandas as pd
import numpy as np
import logging
from pathlib import Path
from typing import Literal, Optional

import pandas as pd
from entities import EM_CDS_TRACKER_DICT, FX_TRACKER_DICT
from utils import Backtest, load_trackers, clean_stale_prices
from pathlib import Path
from aamalha.utils.performance import Performance

c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


In [ ]:
fx = load_trackers(FX_TRACKER_DICT).rename(columns=lambda col: col + "_fx")
cds = load_trackers(EM_CDS_TRACKER_DICT).rename(columns=lambda col: col + "_cds")

trackers = pd.concat(
    [fx, cds],
    axis=1,
).fillna(method="ffill")
trackers = pd.concat(
    [clean_stale_prices(trackers[col].copy()) for col in trackers], axis=1
)
trackers = trackers[:"2024-09-30"]
trackers

In [ ]:
import matplotlib.pyplot as plt 
pairs = [
    "BRL",
    "ZAR",
    # "CNY",
    "COP",
    # "IDR",
    "MXN",
    # "MYR",
    # "RUB",
    "CLP",
    "TRY",
]
N_DAYS = 21
HALFLIFE = 252

signals = {}
data = {}
for code in pairs:
    fx = trackers[f"{code}_fx"].copy()
    cds = trackers[f"{code}_cds"].copy()
    _index = fx.dropna().index.intersection(cds.dropna().index)
    fx = fx.reindex(_index)
    fx = fx / fx.iloc[0]
    cds = cds.reindex(_index)
    cds = cds / cds.iloc[0]
    fx = np.log(fx).diff(1).dropna()
    cds = np.log(cds).diff(1).dropna()
    beta = fx.ewm(halflife=HALFLIFE).cov(cds) / cds.ewm(halflife=HALFLIFE).var()
    beta = beta.iloc[HALFLIFE:].reindex(beta.index)
    actual_fx_returns = (np.exp(fx.rolling(HALFLIFE).sum()) - 1)
    expect_fx_returns = ((np.exp(cds.rolling(HALFLIFE).sum()) - 1) * beta)
    alpha = actual_fx_returns - expect_fx_returns
    actual_fx_returns.plot()
    expect_fx_returns.plot()
    alpha.plot()
    plt.title(code)
    plt.show()
    forward_return = np.exp(fx.rolling(21).sum()) - 1
    # signal = (alpha > 0).map({True: -1, False: 1}).shift(21) * forward_return
    # signals[code] = signal
    data[code] = (alpha, forward_return)

In [ ]:
df_signals = pd.concat(signals.values(), axis=1)
df_signals.columns = signals.keys()
df_signals.mean(axis=1).mean()

performar uma regress~ao dos sign("alpha") (exog) nos retornos de 1 mês
"forward_return" (endog),
sendo que temos timeseries desses dados e para 8 economias ("pairs") 


pairs = [
    "BRL",
    "ZAR",
    "CNY",
    "COP",
    "IDR",
    "MXN",
    "MYR",
    "RUB",
    "CLP",
    "TRY",
]

In [ ]:
sub_dfs = []
for code, (alpha, fwd_return) in data.items():
    # alpha = data["BRL"][0]
    alpha.name = "alpha"
    # fwd_return = data["BRL"][1]
    fwd_return.name = "fwd_return"
    base = pd.concat([alpha, fwd_return], axis=1)
    base["country"] = code
    # base = base.set_index("country", append=True)
    sub_dfs.append(base)  

base = pd.concat(sub_dfs)
base = base.dropna(how="any")
base["sign"] = (base["alpha"] > 0).map({True: -1, False: 1})
# base = base.drop("alpha", axis=1)
base

In [ ]:
# base.pivot(columns=["country"], values=["fwd_return", "sign"])#.dropna(how="any", axis=1)

In [ ]:
base

In [ ]:
import statsmodels.api as sm

X = base["alpha"]  # Variável independente
y = base["fwd_return"]  # Variável dependente

# Adicionar uma constante à variável independente para o intercepto
X = sm.add_constant(X)

# Ajustar o modelo de regressão
modelo = sm.OLS(y, X).fit()

# Exibir os resultados
print(modelo.summary())

In [ ]:
from linearmodels.panel import PanelOLS
base = base.reset_index()

modelo = PanelOLS(
    dependent=base[["fwd_return"]],
    exog=base[["sign"]],
    time_effects=True,
    entity_effects=True,
)


# # Especificar o modelo com efeitos fixos por entidade (country) e tempo
# modelo = PanelOLS.from_formula(
#     "fwd_return ~ sign + EntityEffects + TimeEffects", data=base
# )

# Ajustar o modelo
resultados = modelo.fit()

# Exibir os resultados
print(resultados.summary)